In [1]:
# Basic libraries
import numpy as np
import tensorflow as tf
# LSTM-autoencoder
from LSTMAutoencoder import *
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler


In [2]:
tf.reset_default_graph()
tf.set_random_seed(2016)
np.random.seed(2016)

In [3]:
# Constants
batch_num = 120
hidden_num = 12
step_num = 15
elem_num = 1
iteration = 10000

In [4]:
# load the dataset
dataframe = read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv', usecols=[1], engine='python', skipfooter=3)
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataframe.values.astype('float32'))

data=np.zeros((batch_num,step_num,1))
for b in range(batch_num):
    data[b]=dataset[b:b+step_num]
    
# (batch_num, 8, 1)
 
print(data.shape)
    

(120, 15, 1)


In [5]:
# placeholder list
p_input = tf.placeholder(tf.float32, shape=(batch_num, step_num, elem_num))
p_inputs = [tf.squeeze(t, [1]) for t in tf.split(p_input, step_num, 1)]

cell = tf.nn.rnn_cell.LSTMCell(hidden_num, use_peepholes=True)
ae = LSTMAutoencoder(hidden_num, p_inputs, cell=cell, decode_without_input=True)

Instructions for updating:
Use the retry module or similar alternatives.


In [6]:
# Encapsuler ça dans une fonction au format Train

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iteration):
        iteration_data = np.copy(data)
        np.random.shuffle(iteration_data)
        (loss_val, _) = sess.run([ae.loss, ae.train], {p_input: iteration_data})
        if i%(iteration/20) == 0:
            print('iter %d:' % (i), loss_val)

    (input_, output_) = sess.run([ae.input_, ae.output_], {p_input: data})
    print('train result :')
    print('input :', input_[0, :, :].flatten())
    print('output :', output_[0, :, :].flatten())

iter 0: 0.064719364
iter 500: 0.0057418332
iter 1000: 0.00570651
iter 1500: 0.005690402
iter 2000: 0.005658193
iter 2500: 0.005293341
iter 3000: 0.0029339513
iter 3500: 0.002063871
iter 4000: 0.0017462947
iter 4500: 0.0016550423
iter 5000: 0.0013471859
iter 5500: 0.001169667
iter 6000: 0.0010826061
iter 6500: 0.0010036092
iter 7000: 0.0009477226
iter 7500: 0.0011320115
iter 8000: 0.00089979544
iter 8500: 0.0008475579
iter 9000: 0.0007544717
iter 9500: 0.00076903705
train result :
input : [0.01544401 0.02702703 0.05405405 0.04826255 0.03281853 0.05984557
 0.08494207 0.08494207 0.06177607 0.02895753 0.         0.02702703
 0.02123553 0.04247104 0.07142857]
output : [0.0137329  0.01590722 0.02815244 0.04423974 0.05882653 0.06836172
 0.07021444 0.06305408 0.04943778 0.0360296  0.0297543  0.03342482
 0.04322087 0.05037197 0.05907565]
